# WEBINAR - Przetwarzanie języka naturalnego, 16 lipca 2020

SAGES - NLP masterclass

Łukasz Kobyliński, Ryszard Tuora


2 modele do języka polskiego w spaCy:

IPI PAN - bardziej rozbudowany, wolniejszy, bardziej złożona instalacja - http://zil.ipipan.waw.pl/SpacyPL

model oficjalny - prostszy, znacznie szybszy, prosta instalacja - https://spacy.io/models/pl

model IPI PAN dla języka polskiego

składa się z:
- taggera morfosyntaktycznego
- lematyzatora
- parsera zależnościowego
- komponentu NER
- flexera (komponentu do fleksji)

In [ ]:
# Przygotowanie środowiska, komendy linux
# instalacja Morfeusza 2
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2

# upgrade keras'a
!python3 -m pip install --upgrade keras==2.3.1

# instalacja spaCy

!python3 -m pip install spacy==2.3.0

# 1. instalacja modelu IPI PAN dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz' pl_spacy_model_morfeusz-0.1.0.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.0.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

# 2. instalacja oficjalnego modelu spaCy
!python3 -m spacy download pl_core_news_lg

# dodatkowe zależności:
!python3 -m pip install tqdm

# PO WYKONANIU NALEŻY ZRESETOWAĆ RUNTIME

**Po wykonaniu powyższej komórki należy zresetować runtime**

In [ ]:
### PYTHON 3
import keras
print(keras.__version__) # Powinno wyświetlić 2.3.1
# ładowanie modelu
import spacy
print(spacy.__version__)

nlp = spacy.load("pl_spacy_model_morfeusz")
#nlp = spacy.load("pl_core_news_lg")

# Część zero - Tokenizacja i reprezentacja tekstów

Wejściem do modelu są łańcuchy tekstowe (stringi), na wyjściu dostajemy obiekt reprezentujący struktury wykryte w tekście. Pierwszym krokiem który musi być wykonany aby przetworztć tekst, jest tokenizacja, czyli podział tekstu na tokeny/segmenty. Tokeny w większości przypadków odpowiadają słowom "od spacji do spacji". Ale warto zwrócić uwagę na kilka przypadków odstających od takiej prostej reguły.

Wynikiem potoku spaCy jest obiekt Doc, który składa się z obiektów Token.

In [ ]:
txt = "Chciałby, żebym pojechał do miasta z zielono-żółto-białą flagą (np. Zielonej Góry)."
split = txt.split()
doc = nlp(txt)
print("spaCy          .split()\n")
for i in range(max([len(split), len(doc)])):
  try:
    tok1 = doc[i]
  except IndexError:
    tok1 = ""
  try:
    tok2 = split[i]
  except IndexError:
    tok2 = ""
  print("{0:15} {1:15}".format(tok1.orth_, tok2))

# Część pierwsza - Tagowanie morfosyntaktyczne
korzystamy z tagsetu NKJP
Nasz tagger to słownikowy tagger Morfeusz2 + dezambiguacja za pomocą neuronowego Toyggera (LSTM)

Każdy token t ma trzy interesujące nas atrybuty: 
- t.tag_ : klasa gramatyczna według polskiego tagsetu NKJP (http://nkjp.pl/poliqarp/help/ense2.html)
- t.pos_ : klasa gramatyczna według międzynarodowego tagsetu UD (mapowana z NKJP)
- t._.feats : customowy atrybut odpowiadający cechom morfosyntaktycznym (np. rodzajowi gramatycznemu, lub liczbie), poszczególne wartości cech są oddzielone dwukropkiem

In [ ]:
txt = "Nornica prowadzi zmierzchowo-nocny tryb życia, ale wychodzi również za dnia w poszukiwaniu pokarmu."
doc = nlp(txt) # przetworzenie textu przez pipeline, na wyjściu dostajemy iterowalny obiekty klasy Doc, przechowujący tokeny
for t in doc:
  print("{0:15} {1:8} {2:6} {3:15}".format(t.orth_, t.tag_, t.pos_, t._.feats)) # wypisujemy interpretację morfosyntaktyczną każdego tokenu

##Zadanie 1:


![alt text](https://github.com/ryszardtuora/webinar_resources/raw/master/czesci_mowy.png)

Źródło: Irena Kamińska-Szmaj, *Różnice leksykalne między stylami funkcjonalnymi polszczyzny pisanej: Analiza statystyczna na materiale słownika frekwencyjnego*, 1990.

Przetwórz tekst znajdujący się w zmiennej txt, oblicz proporcję czasowników (używając tagów UPOS), na podstawie tego oszacuj gatunek do którego należy tekst.

In [ ]:
import requests

txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/webinar_resources/master/1.txt").text

#TODO#

### Fleksja

Flexer pozwala na odmianę pojedynczych tokenów, do pożądanej charakterystyki morfologicznej. Argumentami do flexera jest słowo do odmiany (string, lub lepiej otagowany token), i przedzielony dwukropkami string znaczników morfosyntaktycznych.

Flexer umożliwia np. wypełnianie szablonów tekstów.

In [ ]:
filizanka = nlp("filiżanka")[0]
print(filizanka._.feats)
flexer = nlp.get_pipe("flexer")

tmpl1 = "Szukam szarej, porcelanowej {}.".format(flexer.flex(filizanka, "gen"))
tmpl2 = "Marzę o szarej, porcelanowej {}.".format(flexer.flex(filizanka, "loc"))
tmpl3 = "Szukam kompletu porcelanowych {}.".format(flexer.flex(filizanka, "gen:pl"))
print(tmpl1)
print(tmpl2)
print(tmpl3)

#Część druga - Lematyzacja i własności leksykalne
Nasz model umożliwia słownikową lematyzację przy pomocy Morfeusza, do dezambiguacji (tutaj np. rozróżnienia między "mamy"-> "mama" i "mamy" -> "mieć" służy output taggera).

Lematyzacja pozwala redukować redundancję informacyjną, i ułatwiać zadania takie jak streszczanie, i przeszukiwanie.

Każdy z tokenów dodatkowo jest oznaczony ze względu na pewne własności leksykalne, np. :
- t.is_stop - słowo należy do stoplisty (listy słów występujących najczęściej, a więc najmniej istotnych semantycznie)
- t.is_oov - słowo znajduje się poza słownikiem, i.e. embeddingami wykorzystanymi w modelu
- t.like_url - token ma strukturę url-a
- t.like_num - token jest liczbą
- t.is_alpha - token składa się tylko ze znaków alfabetycznych
- t.rank - miejse w rankingu częstości słów

In [ ]:
txt = "Zdenerwowany gen. Leese mówił przez telefon swym podwładnym walczącym pod Monte Cassino, że rozmawia z nimi ze schronu."
doc = nlp(txt)
print("{0:16} {1:16} {2:5} {3:5} {4:20}\n".format("forma", "lemat", "OOV", "STOP", "Częstość"))
for t in doc:
  print("{0:16} {1:16} {2:5} {3:5} {4:20}".format(t.orth_, t.lemma_, t.is_oov, t.is_stop, t.rank)) # orth_ to atrybut odpowiadający formie słowa występującej w tekście

##Zadanie 2:

Przetwórz tekst spod zmiennej txt, przekonwertuj go do listy lematów, usuń słowa ze stoplisty, oraz interpunkcję, wypisz dziesięć najczęściej pojawiających się lematów. Wypróbuj także opcję w której uwzględniamy tylko rzeczowniki


In [ ]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/webinar_resources/master/2.txt").text

#TODO#

#Część trzecia - parsowanie zależnościowe

spaCy zawiera parser zależnościowy oparty o metodologię transition-based dependency parsing. 
Interesują nas tu cztery atrybuty:
 - t.head - link do tokenu będącego nadrzędnikiem tokenu t
 - t.dep_ - etykieta opisująca rodzaj zależności
 - t.subtree - generator opisujący poddrzewo rozpięte przez token t
 - t.ancestors - generator opisujący wszystkie przechodnie nadrzędniki tokenu t


Opis systemu etykiet: https://universaldependencies.org/u/dep/all.html

In [ ]:
txt = "Pierwsza wzmianka o Gdańsku pochodzi ze spisanego po łacinie w 999 Żywotu świętego Wojciecha."
doc = nlp(txt)

import pandas as pd

table = []
for tok in doc:
  tok_dic = {"form": tok.orth_, "label": tok.dep_, "head": tok.head.orth_, "subtree": list(tok.subtree), "ancestors": list(tok.ancestors)}
  table.append(tok_dic)
df = pd.DataFrame(table)
print(df.to_string())

### spaCy posiada wbudowaną wizualizację drzew zależnościowych

In [ ]:
from spacy import displacy

displacy.render(doc, jupyter = True)

###Poniższa funkcja służy łatwej wizualizacji podstawowych własności tokenów z danego tekstu

In [ ]:
import pandas as pd

def table(doc):
  table = []
  for tok in doc:
    tok_dic = {"form": tok.orth_, "lemma": tok.lemma_, "tag": ":".join([tok.tag_, tok._.feats]), "dep_label": tok.dep_, "dep_head": tok.head.orth_}
    table.append(tok_dic)
  return pd.DataFrame(table)

txt = "Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.\
 Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych \
 wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii \
 (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka\
  będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki \
  polskiej. Nożyce między oboma krajami będą się więc rozwierać."

doc = nlp(txt)

tab = table(doc)

print(tab.to_string()) # prosty hack na wypisanie całej tabeli

###Czasami interesują nas większe całości niż pojedyncze tokeny, np. rzeczowniki często łączą się z przymiotnikami w frazy, żeby znajdować takie całości w tekście możemy korzystać z Matchera.

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
doc.is_tagged = True # tymczasowy hack, będzie zbędny w kolejnej wersji modelu
pattern = [{"TAG": "adj"}, {"TAG": "subst"}]
matcher.add("AdjSubst", None, pattern) # nazwa, funkcja, wzór

for match_id, start, end in matcher(doc):
  print(doc[start:end])

#### Niemniej w języku polskim, ze względu na swobodę szyku, takie rozwiązanie jest mało owocne, przymiotnik nie musi zawsze poprzedzać rzeczownika który określa, tego typu zależności są widoczne dopiero na poziomie analizy gramatycznej

### W gramatykach zależnościowych możliwa jest częściowa rekonstrukcja fraz rzeczownikowych (Noun Phrase - NP)
Aby to zrobić, musimy wydobyć z tekstu wszystkie rzeczowniki, zebrać wszystkie rozpinane przez nie poddrzewa, i wyrzucić ze zbioru te poddrzewa, które są już podfrazą większej frazy rzeczownikowej. To frazy rzeczownikowe, a nie same rzeczowniki, są jednostkami które mają dobrze określone znaczenie. Np. w zdaniu:

###*Żona Pawła jest blondynką.* 

nie mówimy o Pawle, ani o jakiejś abstrakcyjnej żonie, tylko o konkretnej kobiecie, którą wyodrębniamy przez fakt że jest żoną Pawła.

##Zadanie 3:

Napisz funkcję która na wejściu pobiera dokument, a na wyjściu zwraca listę fraz rzeczownikowych zgodnie z podanym powyżej opisem. Następnie wyodrębnij wszystkie frazy rzeczownikowe z tekstu pod zmienną txt.

In [ ]:
#TODO#

###Parser zależnościowy pozwala także na dzielenie dokumentów na zdania w sposób bardziej inteligentny, niż posługując się regułami interpunkcji.
###Za zdanie uważamy nieprzerwaną sekwencję tokenów które są powiązane relacjami zależnościowymi.
###Zdania są zapisane w atrybucie doc.sents dokumentu.

In [ ]:
for s in doc.sents:
  print(s)

displacy.render(doc, jupyter = True)

#Część czwarta - Rozpoznawanie jednostek nazewniczych (NER)
####Nasz model do spaCy wykorzystuje 6 rodzajów etykiet:
- placeName - miejsca antropogeniczne, np. Dania, Londyn
- geogName - naturalne miejsca geograficzne, np. Tatry, Kreta
- persName - imiona i nazwiska osób, np. J. F. Kennedy, gen. Maczek
- orgName - nazwy organizacji, np. NATO, Unia Europejska
- date - daty, np. 22 marca 1999, druga połowa kwietnia
- time - czas, np. 18:55, pięć po dwunastej

####Nie pozwala na wykrywanie zagnieżdżonych jednostek nazewniczych, np. [placeName: **aleja** [persName: **Piłsudskiego**]]

####Wykryte wzmianki są przechowywane w atrybucie doc.ents dokumentu, każda z tych wzmianek ma atrybut ent.label_, w którym przechowywana jest jej etykieta.

In [ ]:
print(doc, "\n\n")
for e in doc.ents:
  print(e, e.label_)


###displaCy pozwala także na wizualizację jednostek nazewniczych

In [ ]:
displacy.render(doc, style="ent", jupyter = True)

W obecnym modelu NER, uwzględnione są "uniwersalne" kategorie jednostek nazewniczych, jednak w zależności od zastosowania będziemy najprawdopodobniej potrzebowali innych kategorii - np. osobnej kategorii dla nazw aktów prawodawczych, lub kwot i walut.

Bez problemu można zastąpić domyślny model własnym, wytrenowanym przez CLI spaCy na własnych oznakowanych danych. Więcej informacji tu: https://spacy.io/api/cli#train

##Zadanie 4:

W zmiennej txt znajduje się dokument historyczny, wydobądź z niego wszystkie *zdania* zawierające daty, i po kolei je wypisz, rozważ możliwe sposoby automatycznego szeregowania dat wyrażonych w różny sposób (po ewentualnym sprowadzeniu ich do kanonicznej postaci). Rozwiązanie tego problemu jest trudne, i istnieją do niego odrębne narzędzia, umożliwiają one np. automatyczną rekonstrukcję chronologii wydarzeń.

In [ ]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/webinar_resources/master/3.txt").text

#TODO#

# Część piąta - podobieństwo w języku
Większość powyższych metod opiera się o embeddingi, czyli funkcje przypisujące słowom wektory w przestrzeni wielowymiarowej (najczęściej 100 lub 300-wymiarowej). Wektory te nie są przypisywane arbitralnie, lecz własności tej przestrzeni mają w jakiś sposób odzwierciedlać własności języka. Np. odległość między wektorami powinna odpowiadać odległości między słowami, którą możemy rozumieć jako podobieństwo znaczeń.

###Bardzo popularny przykład "arytmetyki słów":
znaczenie słów jest reprezentowane przez wektory, dla których mamy zdefiniowane operacje matematyczne. Możemy więc odjąć od znaczenia słowa "królowa", znaczenie słowa "kobieta", i dodać doń znaczenie słowa "mężczyzna", licząc iż wektor będący wynikiem takiego działania odpowiada słowu "król". W praktyce wektor taki najprawdopodobniej nie ma interpretacji, ale możemy znaleźć najbliższy wektor który ma jakąkolwiek interpretację przeszukując słownik.

In [ ]:
from scipy import spatial
 
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['mężczyzna'].vector
woman = nlp.vocab['kobieta'].vector
queen = nlp.vocab['królowa'].vector
king = nlp.vocab['król'].vector
 
# We now need to find the closest vector in the vocabulary to the result of "man" - "woman" + "queen"
maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue
 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in computed_similarities[:10]])

###spaCy umożliwia liczenie podobieństwa między słowami 
####jest ono proporcjonalne do cosinusa kąta wektorami je reprezentującymi. Odpowiednia funkcja jest zdefiniowana dla leksemów (tutaj oznaczają one słowa z pominięciem kontekstu).

In [ ]:
w1 = "pies"
w2 = "psami"
w3 = "zwierzę"
w4 = "buldog"
w5 = "obroża"
w6 = "smycz"
w7 = "marchewka"
w8 = "słońce"

def similarity(w1, w2):
  w1_lex = nlp.vocab[w1]
  w2_lex = nlp.vocab[w2]
  if w1_lex.has_vector and w2_lex.has_vector:
    sim = w1_lex.similarity(w2_lex)
    print("{} vs. {} -> {}".format(w1, w2, sim))
  else:
    print("Jedno ze słów nie ma reprezentacji wektorowej.")
for w in [w2, w3, w4, w5, w6, w7, w8]:
  similarity(w1, w)

## Zadanie 5:

nlp.vocab może być traktowany jako iterator (stanowi słownik, choć nie wszystkim słowom przypisane są wektory). Napisz funkcję thesaurus(word) która dla podanego słowa znajduje dziesięć najbardziej podobnych słów. Pamiętaj o złożoności obliczeniowej sortowania! 

Możesz wykorzystać lematyzację, oraz tagowanie morfosyntaktyczne, by pozbyć się zbędnych słów.

In [ ]:
#TODO#